In [64]:
#Imports
import pandas as pd
from datetime import datetime
from g2p_en import G2p
#pip install g2p_en <- erforderlich für die Phonemtranskribierung
#oder python setup.py install

In [65]:
#CSV einlesen und Dataframes erstellen
cols_twistlist = ['text', 'phonemes', 'keywords', 'source']
twistList = pd.read_csv(r'data source 2\raw-data\raw-data\twister_dataset.csv', delimiter=',',nrows= 3000,header=None, names=cols_twistlist)
cols_kaggle = ['0', 'text']
kaggle = pd.read_csv(r'data source 2\raw-data\raw-data\datasetKaggle.csv', delimiter=',',nrows= 1000,header=None, usecols=['text'], names=cols_kaggle)
twistList

,text,phonemes,keywords,source
0,tongue-twister,phonetic-transcription,keywords,source
1,Peter Piper picked a peck of pickled peppers. ...,[P IY1 T ER0] [P AY1 P ER0] [P IH1 K T] [AH0] ...,pickled peppers peter piper picked,https://www.kaggle.com/datasets/djathidiro/ton...
2,I saw Susie sitting in a shoe shine shop. Wher...,[AY1] [S AO1] [S UW1 Z IY0] [S IH1 T IH0 NG] [...,shoe shine shop,https://www.kaggle.com/datasets/djathidiro/ton...
3,How many boards Could the Mongols hoard If the...,[HH AW1] [M EH1 N IY0] [B AO1 R D Z] [K UH1 D]...,mongol hordes got bored,https://www.kaggle.com/datasets/djathidiro/ton...
4,How can a clam cram in a clean cream can?,[HH AW1] [K AE1 N] [AH0] [K L AE1 M] [K R AE1 ...,clean cream,https://www.kaggle.com/datasets/djathidiro/ton...
...,...,...,...,...
2598,Smelly Sally sells soft satin sheep sheets.,[S M EH1 L IY0] [S AE1 L IY0] [S EH1 L Z] [S A...,smelly sally sells soft satin sheep sheets,https://archive.org/details/reallyreallygros00...
2599,Silly Sally Smits sold seven short salesmen sm...,[S IH1 L IY0] [S AE1 L IY0] [S M IH1 T S] [S O...,silly sally smits sold seven short salesmen sm...,https://archive.org/details/reallyreallygros00...
2600,Do Dewey and Didi do dizzy dances daily?,[D UW1] [D UW1 IY0] [AH0 N D] [D IY1 D IY0] [D...,dizzy dances daily,https://archive.org/details/reallyreallygros00...
2601,Please place Petey Pooh's purple porta potty b...,[P L IY1 Z] [P L EY1 S] [P IY1 T IY0] [P UW1 Z...,please place petey pooh,https://archive.org/details/reallyreallygros00...


In [66]:
#Beide DataFrames "zuschneiden"
dfTL = twistList.drop(['keywords', 'source'], axis=1)

dfKaggle = kaggle.copy()

print(dfTL.shape)
print(dfKaggle.shape)

(2603, 2)
(606, 1)


In [67]:
#dfKaggle in Phoneme übersetzen

dfKaggleP = [] #<- Liste für Phonemrepräsentation
g2p = G2p()
for index, row in dfKaggle.iterrows():
    text = row['text']
    dfKaggleP.append(g2p(text))
# print(dfKaggleP)
print(len(dfKaggleP))

dfKaggle['phonemes'] = dfKaggleP

duplic = dfKaggle[~dfKaggle['text'].isin(dfTL['text'])] #<- nur die Zungenbrecher, welche noch nicht in der TwistList vorkommen



606


In [68]:
print(len(dfKaggleP))
print(dfKaggle.shape)

print(duplic.shape)
duplic

606
(606, 2)
(75, 2)


,text,phonemes
0,0,"[Z, IH1, R, OW0]"
21,"To sit in solemn silence in a dull, dark dock,...","[T, UW1, , S, IH1, T, , IH0, N, , S, AA1, L..."
29,"I scream, you scream, we all scream for icecream!","[AY1, , S, K, R, IY1, M, , ,, , Y, UW1, , ..."
34,"Celibate celebrant, celibate celebrant, celiba...","[S, EH1, L, IH0, B, AH0, T, , S, EH1, L, AH0,..."
41,"What a terrible tongue twister, what a terribl...","[W, AH1, T, , AH0, , T, EH1, R, AH0, B, AH0,..."
...,...,...
561,Then step up mister and twist your tongue,"[DH, EH1, N, , S, T, EH1, P, , AH1, P, , M,..."
562,Now Kissle will whistle at busty Miss. Russell...,"[N, AW1, , K, IH1, S, AH0, L, , W, IH1, L, ..."
576,"Purple paper people, purple paper people, purp...","[P, ER1, P, AH0, L, , P, EY1, P, ER0, , P, I..."
577,De doorgaans dappere Durgerdammer drukker Dirk...,"[D, IY1, , D, AO1, R, G, AH0, N, Z, , D, AE1..."


In [72]:
#Dataframes zusammenführen

print(dfTL.shape)
print(dfTL.columns)

print(duplic.shape)
print(duplic.columns)


finDF = dfTL.append(duplic, ignore_index=True)



print(finDF.shape)

(2603, 2)
Index(['text', 'phonemes'], dtype='object')
(75, 2)
Index(['text', 'phonemes'], dtype='object')
(2678, 2)


C:\Users\Alberto\AppData\Local\Temp\ipykernel_18504\3000386190.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finDF = dfTL.append(duplic, ignore_index=True)


In [ ]:
#Satzzeichen aus Phonemen entfernen
#finDF = finDF[1].str.replace(r'[\[!]]','',regex=True)
#print(finDF)

In [ ]:
#DataFrame als .csv exportieren
date = datetime.now().strftime('%Y-%m-%d')
filename = f'{date}_Dataset.csv'
finDF.to_csv(filename, index= False)

NameError: name 'finDF' is not defined